# Completa tabla de ventas con 0 (ceros)

In [1]:
# EJECUTA LIBRERIAS
import duckdb
import os
import pandas as pd

In [2]:
# LEEMOS LOS DATOS
drive_base_path = 'C:/Repositorios-Ing.Carlos-Cicconi/labo3-2025r/Data/'
filename = 'sell-in.txt'
filepath = os.path.join(drive_base_path, filename)
sell = pd.read_csv(filepath, sep='\t')
print(sell.head(10))
print(f"Datos leídos: {sell.shape[0]} filas y {sell.shape[1]} columnas")
print(f"Valores posibles plan precios cuidados: {sell['plan_precios_cuidados'].unique()}")

   periodo  customer_id  product_id  plan_precios_cuidados  cust_request_qty  \
0   201701        10234       20524                      0                 2   
1   201701        10032       20524                      0                 1   
2   201701        10217       20524                      0                 1   
3   201701        10125       20524                      0                 1   
4   201701        10012       20524                      0                11   
5   201701        10080       20524                      0                 1   
6   201701        10015       20524                      0                 4   
7   201701        10062       20524                      0                 1   
8   201701        10159       20524                      0                 3   
9   201701        10183       20524                      0                 1   

   cust_request_tn       tn  
0          0.05300  0.05300  
1          0.13628  0.13628  
2          0.03028  0.03028  

In [3]:
# Determina cantidades de cada variable
# Periodos
periods = pd.DataFrame({'periodo': sell['periodo'].unique()})
print(f"Cant. periodos: {periods.shape[0]}")
# Clientes
customers = pd.DataFrame({'customer_id': sell['customer_id'].unique()})
print(f"Cant. clientes: {customers.shape[0]}")
# Productos
products = pd.DataFrame({'product_id': sell['product_id'].unique()})
print(f"Cant. productos: {products.shape[0]}")

Cant. periodos: 36
Cant. clientes: 597
Cant. productos: 1233


In [4]:
# Genera dataset prod_born: primer periodo de aparición de cada product_id
prod_born = (
    sell.groupby('product_id', as_index=False)['periodo']
    .min()
)

# Genera dataset cust_born: primer periodo de aparición de cada customer_id
cust_born = (
    sell.groupby('customer_id', as_index=False)['periodo']
    .min()
)

# Genera dataset prod_dead: último periodo de aparición de cada product_id
prod_dead = (
    sell.groupby('product_id', as_index=False)['periodo']
    .max()
)

In [5]:
# 1. Crear todas las combinaciones posibles usando DuckDB
con = duckdb.connect()
con.register('periodos', periods)
con.register('clientes', customers)
con.register('productos', products)
con.register('sell', sell)

query = """
SELECT 
    p.periodo, 
    c.customer_id, 
    pr.product_id
FROM periodos p
CROSS JOIN clientes c
CROSS JOIN productos pr
"""
combinaciones = con.execute(query).df()
print(f"Cant. combinac. TOTALES: {combinaciones.shape[0]}")

Cant. combinac. TOTALES: 26499636


In [6]:
# 2. Hacer LEFT JOIN con sell para traer los valores, usando DuckDB
con.register('combinaciones', combinaciones)

query_full = """
SELECT 
    comb.periodo,
    comb.customer_id,
    comb.product_id,
    COALESCE(s.plan_precios_cuidados, 0) AS plan_precios_cuidados,
    COALESCE(s.cust_request_qty, 0) AS cust_request_qty,
    COALESCE(s.cust_request_tn, 0) AS cust_request_tn,
    COALESCE(s.tn, 0) AS tn
FROM combinaciones comb
FULL OUTER JOIN sell s
    ON comb.periodo = s.periodo
    AND comb.customer_id = s.customer_id
    AND comb.product_id = s.product_id
"""

dataset_full = con.execute(query_full).df()

In [7]:
# Unir dataset_full con prod_born para traer el periodo de aparición de cada product_id
dataset_full = dataset_full.merge(prod_born, on='product_id', how='left', suffixes=('', '_born'))
# Filtrar: solo dejar los registros donde periodo >= periodo_born
dataset_full = dataset_full[dataset_full['periodo'] >= dataset_full['periodo_born']]
# (Opcional) Eliminar la columna auxiliar periodo_born
dataset_full = dataset_full.drop(columns=['periodo_born'])

In [8]:
# Unir dataset_full con prod_dead para traer el último periodo de cada product_id
dataset_full = dataset_full.merge(prod_dead, on='product_id', how='left', suffixes=('', '_dead'))
# Filtrar: solo dejar los registros donde periodo <= periodo_dead
dataset_full = dataset_full[dataset_full['periodo'] <= dataset_full['periodo_dead']]
# (Opcional) Eliminar la columna auxiliar periodo_dead
dataset_full = dataset_full.drop(columns=['periodo_dead'])

In [9]:
# Unir dataset_full con cust_born para traer el periodo de aparición de cada customer_id
dataset_full = dataset_full.merge(cust_born, on='customer_id', how='left', suffixes=('', '_born'))
# Filtrar: solo dejar los registros donde periodo >= periodo_born
dataset_full = dataset_full[dataset_full['periodo'] >= dataset_full['periodo_born']]
# (Opcional) Eliminar la columna auxiliar periodo_born
dataset_full = dataset_full.drop(columns=['periodo_born'])

In [10]:
# Exportar el dataset depurado
print(f"Valores posibles plan precios cuidados: {dataset_full['plan_precios_cuidados'].unique()}")

drive_base_path = 'C:/Repositorios-Ing.Carlos-Cicconi/labo3-2025r/Outputs/completa ceros/'
filename = 'sell_in_con_ceros.csv'
filepath = os.path.join(drive_base_path, filename)
dataset_full.to_csv(filepath, index=False)

Valores posibles plan precios cuidados: [0 1]


In [11]:
# LEEMOS LOS DATOS PARA 'tb_productos.txt' y para 'tb_stocks.txt'
drive_base_path = 'C:/Repositorios-Ing.Carlos-Cicconi/labo3-2025r/Data/'
filename = 'tb_productos.txt'
filepath = os.path.join(drive_base_path, filename)
tb_productos = pd.read_csv(filepath, sep='\t')

filename = 'tb_stocks.txt'
filepath = os.path.join(drive_base_path, filename)
tb_stocks = pd.read_csv(filepath, sep='\t')

In [12]:
dataset_full_sell_prod = dataset_full.merge(tb_productos, 
                       on='product_id', 
                       how='left')

In [13]:
dataset_full_sell_prod_stk = dataset_full_sell_prod.merge(tb_stocks, 
                       on=['product_id', 'periodo'], 
                       how='left')

drive_base_path = 'C:/Repositorios-Ing.Carlos-Cicconi/labo3-2025r/Outputs/completa ceros/'
filename = 'sell_in_con_ceros_merged.csv'
filepath = os.path.join(drive_base_path, filename)

dataset_full_sell_prod_stk.to_csv(filepath, index=False)
print(f"Datos leídos: {dataset_full_sell_prod_stk.shape[0]} filas y {dataset_full_sell_prod_stk.shape[1]} columnas")

Datos leídos: 17173448 filas y 14 columnas
